<a href="https://colab.research.google.com/github/saroshkhawaja/Data-Science-Ecosystem/blob/main/Ticket_Management_System_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#all the complexities are added in this code
import queue

class Ticket:
    """
    Class to represent a Ticket.
    """
    next_ticket_id = 1  # Static variable to store the next ticket ID

    def __init__(self, title, description, category):
        """
        Constructor for the Ticket class. O(1)
        """
        self.ticket_id = Ticket.next_ticket_id  # Assign unique ID and increment
        Ticket.next_ticket_id += 1
        self.title = title
        self.description = description
        self.category = category
        self.status = "Open"  # Initial status

    def __str__(self):
        """
        Returns a string representation of the Ticket object. O(1)
        """
        return f"Ticket ID: {self.ticket_id}\n" \
               f"Title: {self.title}\n" \
               f"Description: {self.description}\n" \
               f"Category: {self.category}\n" \
               f"Status: {self.status}"

    def set_status(self, new_status):
        """
        Sets the status of the ticket. O(1)
        """
        self.status = new_status

class BSTNode:
    """
    Class for a Node in the Binary Search Tree.
    """
    def __init__(self, ticket):
        """
        Constructor for the BSTNode class. O(1)
        """
        self.ticket = ticket
        self.left = None
        self.right = None

class TicketManagementSystem:
    """
    Class to manage the Ticket Management System.
    """
    def __init__(self):
        """
        Constructor for the TicketManagementSystem class. O(1)
        """
        self.open_tickets = queue.Queue()  # Queue for open tickets
        self.resolved_tickets = queue.LifoQueue()  # Stack for resolved tickets (using LifoQueue)
        self.ticket_history = []  # List for all ticket history (Linked List replacement)
        self.ticket_map = {}  # Dictionary for ticket ID lookup (Hash Map replacement)
        self.category_tree_root = None  # Root of the BST for categories

    # Helper Functions (private, for internal use within the class)
    def _insert_into_category_tree(self, root, ticket):
        """
        Function to insert a ticket into the BST.  Recursion is natural for BSTs.
        Best/Average Case: O(log n), Worst Case: O(n)
        """
        if root is None:
            return BSTNode(ticket)
        if ticket.category < root.ticket.category:
            root.left = self._insert_into_category_tree(root.left, ticket)
        elif ticket.category > root.ticket.category:
            root.right = self._insert_into_category_tree(root.right, ticket)
        # No else, we don't allow duplicate categories.
        return root

    def _view_tickets_by_category(self, root):
        """
        Function to traverse and display tickets in a BST (Inorder traversal).
        Recursion is key here. O(n)
        """
        if root is not None:
            self._view_tickets_by_category(root.left)
            print(root.ticket)
            self._view_tickets_by_category(root.right)

    def _search_ticket_by_id(self, id):
        """
        Function to find a ticket by its ID using the hash map. O(1) average, O(n) worst.
        """
        if id in self.ticket_map:
            return self.ticket_map[id]  # Returns the Ticket object.
        else:
            return None # changed to None

    # Public Methods
    def add_ticket(self, title, description, category):
        """
        Function to add a new ticket to the system.  This is the core functionality.
        O(log n) average, O(n) worst
        """
        new_ticket = Ticket(title, description, category)  # Create the Ticket object
        self.open_tickets.put(new_ticket)  # Add to the open tickets queue. O(1)
        self.ticket_history.append(new_ticket)  # Add to the ticket history. O(1)
        self.ticket_map[new_ticket.ticket_id] = new_ticket  # Add to the hash map. O(1)
        self.category_tree_root = self._insert_into_category_tree(self.category_tree_root, new_ticket)  # Add to the BST. O(log n) average, O(n) worst
        print(f"Ticket added successfully. Ticket ID: {new_ticket.ticket_id}")

    def view_open_tickets(self):
        """
        Function to view all open tickets. O(n)
        """
        if self.open_tickets.empty():
            print("No open tickets.")
            return
        print("Open Tickets:")
        temp_queue = queue.Queue() # Create a temporary queue
        while not self.open_tickets.empty():
            ticket = self.open_tickets.get()
            print(ticket)
            temp_queue.put(ticket) #add to temp queue
        while not temp_queue.empty(): #refill the original queue.
            self.open_tickets.put(temp_queue.get())

    def resolve_ticket(self):
        """
        Function to resolve a ticket. O(1)
        """
        if self.open_tickets.empty():
            print("No open tickets to resolve.")
            return
        resolved_ticket = self.open_tickets.get()  # Get the ticket from the front of the queue. O(1)
        resolved_ticket.set_status("Resolved") # O(1)
        self.resolved_tickets.put(resolved_ticket)  # Add it to the resolved tickets stack (using LifoQueue). O(1)
        self.ticket_history.append(resolved_ticket) # O(1)
        self.ticket_map[resolved_ticket.ticket_id] = resolved_ticket #update. O(1)
        print(f"Ticket ID {resolved_ticket.ticket_id} resolved.")

    def view_resolved_tickets(self):
        """
        Function to view all resolved tickets. O(n)
        """
        if self.resolved_tickets.empty():
            print("No resolved tickets.")
            return
        print("Resolved Tickets:")
        temp_stack = queue.LifoQueue()
        while not self.resolved_tickets.empty():
            ticket = self.resolved_tickets.get()
            print(ticket)
            temp_stack.put(ticket)
        while not temp_stack.empty():
            self.resolved_tickets.put(temp_stack.get())

    def search_ticket_by_id(self, id):
        """
        Function to search for a ticket by its ID. O(1)
        """
        found_ticket = self._search_ticket_by_id(id)  # Use the helper function. O(1)
        if found_ticket:
            print(found_ticket)
        else:
            print(f"Ticket with ID {id} not found.")

    def view_tickets_by_category(self, category):
        """
        Function to view tickets by category. O(n)
        """
        print(f"Tickets in Category: {category}")
        if self.category_tree_root is None:
            print("No tickets in this category")
            return
        self._view_tickets_by_category(self.category_tree_root) # O(n)

    def view_ticket_history(self):
        """
        Function to view the full ticket history. O(n)
        """
        if not self.ticket_history:
            print("No ticket history.")
            return
        print("Ticket History:")
        for ticket in self.ticket_history: # Iterates through the list.
            print(ticket)

def display_menu():
    """
    Function to display the main menu.  Keeps the user interface separate from the logic. O(1)
    """
    print("\nTicket Management System Menu")
    print("1. Add New Ticket")
    print("2. View Open Tickets")
    print("3. Resolve Ticket")
    print("4. View Resolved Tickets")
    print("5. Search Ticket by ID")
    print("6. View Tickets by Category")
    print("7. View Full Ticket History")
    print("0. Exit")
    print("Enter your choice: ", end="")

def main():
    """
    Main function - Entry point of the program.
    """
    tms = TicketManagementSystem()  # Create an instance of the TicketManagementSystem

    while True:
        display_menu() # O(1)
        try:
            choice = int(input())
        except ValueError:
            print("Invalid input. Please enter a number.")
            choice = -1  # Set to -1 to loop again
            continue

        if choice == 1:
            title = input("Enter ticket title: ")
            description = input("Enter ticket description: ")
            category = input("Enter ticket category: ")
            tms.add_ticket(title, description, category) # O(log n) average, O(n) worst
        elif choice == 2:
            tms.view_open_tickets() # O(n)
        elif choice == 3:
            tms.resolve_ticket() # O(1)
        elif choice == 4:
            tms.view_resolved_tickets() # O(n)
        elif choice == 5:
            try:
                id = int(input("Enter ticket ID to search: "))
            except ValueError:
                print("Invalid Input")
                continue
            tms.search_ticket_by_id(id) # O(1)
        elif choice == 6:
            category = input("Enter category to view tickets: ")
            tms.view_tickets_by_category(category) # O(n)
        elif choice == 7:
            tms.view_ticket_history() # O(n)
        elif choice == 0:
            print("Exiting program.")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()




Ticket Management System Menu
1. Add New Ticket
2. View Open Tickets
3. Resolve Ticket
4. View Resolved Tickets
5. Search Ticket by ID
6. View Tickets by Category
7. View Full Ticket History
0. Exit
Enter your choice: 0
Exiting program.
